<a href="https://colab.research.google.com/github/wairegistacy/twitterMining/blob/master/scrapping_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Web scrapping using python

#### References
1. [Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
2. [Web Scraping using Python](https://www.datacamp.com/community/tutorials/web-scraping-using-python)

In [ ]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [ ]:
#Better
!pip install requests BeautifulSoup4 fire

  Using cached https://files.pythonhosted.org/packages/34/a7/0e22e70778aca01a52b9c899d9c145c6396d7b613719cd63db97ffa13f2f/fire-0.3.1.tar.gz
  Using cached https://files.pythonhosted.org/packages/8a/48/a76be51647d0eb9f10e2a4511bf3ffb8cc1e6b14e9e4fab46173aa79f981/termcolor-1.1.0.tar.gz
  Stored in directory: /Users/yabebal/Library/Caches/pip/wheels/c1/61/df/768b03527bf006b546dce284eb4249b185669e65afc5fbb2ac
  Stored in directory: /Users/yabebal/Library/Caches/pip/wheels/7c/06/54/bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
Successfully built fire termcolor
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [ ]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [10]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa', tag='h2') 
res
df = pd.DataFrame(res)
df.head(10)

,0
0,100. Jeffrey Gettleman (@gettleman)
1,99. Africa24 Media (@a24media)
2,98. Scapegoat (@andiMakinana)
3,97. Africa Check (@AfricaCheck)
4,96. James Copnall (@JamesCopnall)
5,95. Online Africa (@oafrica)
6,94. Patrick Ngowi (@PatrickNgowi)
7,93. DOS African Affairs (@StateAfrica)
8,92. MoadowAJE (@Moadow)
9,91. Brendan Boyle (@BrendanSAfrica)


In [11]:
# extract the first ten twitter handles 
df = pd.DataFrame(res)
df.tail(15)

,0
90,10. Computicket (@Computicket)
91,9. loyiso gola (@loyisogola)
92,8. 5FM (@5FM)
93,7. mailandguardian (@mailandguardian)
94,6. Helen Zille (@helenzille)
95,5. Julius Sello Malema (@Julius_S_Malema)
96,4. News24 (@News24)
97,3. Jacob G. Zuma (@SAPresident)
98,2. Gareth Cliff (@GarethCliff)
99,1. Trevor Noah (@Trevornoah)


In [21]:
# cleaning the data
df1 = df[0].str.split('(', expand=True)
df1.tail(15)
df2 = df1[1].str.split(')', expand=True)
df2.tail(15)
df3 = df2.dropna(axis=0, how='any')
df3.tail(10) 
df4 = df3.sort_index(ascending=0)
df4.head(10)
df5 = df4[0].replace('@','', regex=True)
df5.head(10)

99         Trevornoah
98        GarethCliff
97        SAPresident
96             News24
95    Julius_S_Malema
94         helenzille
93    mailandguardian
92                5FM
91         loyisogola
90        Computicket
Name: 0, dtype: object

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [29]:
# Fetching twitter data
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

consumer_key = 'EmQbpXWpzQSYLb8eI8Gc4sEk2'
consumer_secret = 'tL5eofcBlQFicqGrTu1LpYcuEKVC49bcduwYr06xUtgw2rKp82'
access_token = '1283764477368860678-mfX2LwpfiKMmUDGIQ8tvEhJ7tDxofj'
access_token_secret = 'bvqw6m2OQzROZv2KCLGHBqQXWnoC6R1J6MuMjdH8I3r3f'
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)
# calling the api
api = tweepy.API(auth)

for target in df5:
    try:
       print("Getting data for " + target)
       item = api.get_user(target)
       print("name: " + item.name)
       print("screen_name: " + item.screen_name)
       print("description: " + item.description)
       print("statuses_count: " + str(item.statuses_count))
       print("friends_count: " + str(item.friends_count))
       print("followers_count: " + str(item.followers_count))
       # Using the API object to get tweets from your timeline, and storing it in a variable called public_tweets
       tweets = api.user_timeline(target, count=5)
       counter = 0
       favcounter= 0
       # foreach through all tweets pulled
       for tweet in tweets:
      # printing the text stored inside the tweet object
        print (tweet.text)
        # fetching the retweet_count attribute 
        retweet_count = tweet.retweet_count  
        print("The number of time the tweet has been retweeted is : " + str(retweet_count)) 
        # fetching the favorite_count attribute 
        favorite_count = tweet.favorite_count 
        print("The number of time the status has been favourited is : " + str(favorite_count)) 
        counter += retweet_count
        favcounter += favorite_count
        popularity_score = counter + favcounter   
        print('The popularity score is: ',popularity_score)
    except tweepy.TweepError :
        pass

Getting data for Trevornoah
name: Trevor Noah
screen_name: Trevornoah
description: Comedian from South Africa. I was in the crowd when Rafiki held Simba over the edge of the cliff, like an African Michael Jackson.
statuses_count: 11185
friends_count: 325
followers_count: 10804117
Actual video footage of me starting off the week! https://t.co/2pt2yh47ry
The number of time the tweet has been retweeted is : 2738
The number of time the status has been favourited is : 20723
The popularity score is:  23461
RT @KingBach: https://t.co/Sj2sVP7Gdt
The number of time the tweet has been retweeted is : 92890
The number of time the status has been favourited is : 0
The popularity score is:  116351
When white people take “being an ally” too far. 😂 https://t.co/65vOEAYx7G
The number of time the tweet has been retweeted is : 6001
The number of time the status has been favourited is : 43358
The popularity score is:  165710
Me at the restaurant after things return to normal. 🙌🏾😂 https://t.co/B8sFz5oXZn
T

In [30]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [42]:
import re
res = get_elements(response, search={'find_all':{'class_':'wp-block-embed__wrapper'}})
res

findaing all of {'class_': 'wp-block-embed__wrapper'}


['\nhttps://twitter.com/TsholetsaDomi/status/1238324860536922112\n',
 '\nhttps://twitter.com/Azali_officiel/status/1239649350747332613\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees' unions to map a collaborative effort in the fight against <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a>. <a href="https://t.co/EIYNGOEKRN">pic.twitter.com/EIYNGOEKRN</a></p>— Eswatini Government (@EswatiniGovern1) <a href="https://twitter.com/EswatiniGovern1/status/1241038139889721346?ref_src=twsrc%5Etfw">March 20, 2020</a></blockquote>,
 <script async="" charset="utf-8" src="https://platform.twitter.com/widgets.js"></script>,
 '\n',
 '\nhttps://twitter.com/SE_Rajoelina/status/1241101811647500288\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">GUIDEL

findaing all of {'class_': 'wp-block-embed__wrapper'}


['\nhttps://twitter.com/TsholetsaDomi/status/1238324860536922112\n',
 '\nhttps://twitter.com/Azali_officiel/status/1239649350747332613\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees' unions to map a collaborative effort in the fight against <a href="https://twitter.com/hashtag/COVID19?src=hash&amp;ref_src=twsrc%5Etfw">#COVID19</a>. <a href="https://t.co/EIYNGOEKRN">pic.twitter.com/EIYNGOEKRN</a></p>— Eswatini Government (@EswatiniGovern1) <a href="https://twitter.com/EswatiniGovern1/status/1241038139889721346?ref_src=twsrc%5Etfw">March 20, 2020</a></blockquote>,
 <script async="" charset="utf-8" src="https://platform.twitter.com/widgets.js"></script>,
 '\n',
 '\nhttps://twitter.com/SE_Rajoelina/status/1241101811647500288\n',
 '\n',
 <blockquote class="twitter-tweet" data-dnt="true" data-width="550"><p dir="ltr" lang="en">GUIDEL

In [44]:
str_cells2=str(res1)
cleantext=BeautifulSoup(str_cells2).get_text()
data=[]
for item in cleantext.split("20,"):
    clean=re.compile(r'@(\w*)')
    clean2=re.search(clean,item)
    if clean2 is not None:
        data.append({'value': clean2.group(0) })
    
df=pd.DataFrame(data)
df.columns = ["African leaders"]
df.head(36)


,African leaders
0,@EswatiniGovern1
1,@MalawiGovt
2,@hagegeingob
3,@FinanceSC
4,@PresidencyZA
5,@Dora_Siliya
6,@edmnangagwa
7,@MinSantedj
8,@hawelti
9,@StateHouseKenya


In [48]:
df1 = df.sort_index(ascending=0)
df1.head(10)
df2 = df1.replace('@','', regex=True)
df2.head(10)

,African leaders
35,MSPS_Togo
34,PresidentBio
33,PaulKagame
32,MBuhari
31,IssoufouMhm
30,CheikhGhazouani
29,PresidenceMali
28,USEmbalo
27,President_GN
26,NAkufoAddo


In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

consumer_key = 'EmQbpXWpzQSYLb8eI8Gc4sEk2'
consumer_secret = 'tL5eofcBlQFicqGrTu1LpYcuEKVC49bcduwYr06xUtgw2rKp82'
access_token = '1283764477368860678-mfX2LwpfiKMmUDGIQ8tvEhJ7tDxofj'
access_token_secret = 'bvqw6m2OQzROZv2KCLGHBqQXWnoC6R1J6MuMjdH8I3r3f'
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# set access to user's access key and access secret
auth.set_access_token(access_token, access_token_secret)
# calling the api
api = tweepy.API(auth)

for target in df2:
    try:
       print("Getting data for " + target)
       item = api.get_user(target)
       print("name: " + item.name)
       print("screen_name: " + item.screen_name)
       print("description: " + item.description)
       print("statuses_count: " + str(item.statuses_count))
       print("friends_count: " + str(item.friends_count))
       print("followers_count: " + str(item.followers_count))
       # Using the API object to get tweets from your timeline, and storing it in a variable called public_tweets
       tweets = api.user_timeline(target, count=5)
       counter = 0
       favcounter= 0
       # foreach through all tweets pulled
       for tweet in tweets:
      # printing the text stored inside the tweet object
        print (tweet.text)
        # fetching the retweet_count attribute 
        retweet_count = tweet.retweet_count  
        print("The number of time the tweet has been retweeted is : " + str(retweet_count)) 
        # fetching the favorite_count attribute 
        favorite_count = tweet.favorite_count 
        print("The number of time the status has been favourited is : " + str(favorite_count)) 
        counter += retweet_count
        favcounter += favorite_count
        popularity_score = counter + favcounter   
        print('The popularity score is: ',popularity_score)
    except tweepy.TweepError :
        pass

Getting data for African leaders


## Web scrapping using bash script
If the web site has a quite simple HTML, you can easily use curl to perform the request and then extract the needed values using bash commands grep, cut , sed, ..

This tutorial is adapted from [this](https://medium.com/@LiliSousa/web-scraping-with-bash-690e4ee7f98d) medium article

In [ ]:
%%bash 

# curl the page and save content to tmp_file
#url = "https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa"
#curl -X GET $url -o tmp_file


#!/bin/bash

# write headers to CSV file
echo "Name, twitter_id" >> extractData.csv
n="1"
while [ $n -lt 2 ]
do
  
  #get title
  title=$(cat tmp_file | grep "class=\"twitter-tweet\"" | cut -d ';' -f1 )
  echo $title
  #get author
  #twitter_id=$(cat tmp_file |grep -A1 "class=\"css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0\"" | tail -1)

  #echo "$title, $twitter_id" >> extractData.csv
  #echo "$title, $twitter_id"
    
  n=$[$n+1]

done

<blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr">The Deputy Prime Minister Themba Masuku has today met representatives of the private sector and employees&#39 <blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr">GUIDELINES FOR SCHOOLS IN <a href="https://twitter.com/hashtag/MALAWI?src=hash&amp <blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr">Fellow Namibians, I declared a State of Emergency on <a href="https://twitter.com/hashtag/COVID19?src=hash&amp <blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr"><a href="https://twitter.com/hashtag/COVID19measuresSC?src=hash&amp <blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr">The Minister for Cooperative Governance &amp <blockquote class="twitter-tweet" data-width="550" data-dnt="true"><p lang="en" dir="ltr">Join the <a href="https://twitter.com/hasht